<a href="https://colab.research.google.com/github/fjadidi2001/Image_Inpaint/blob/main/BIDS_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bidirectional Interaction Dual-Stream Network (BIDS-Net)

> The article proposes a novel approach for image inpainting called the Bidirectional Interaction Dual-Stream Network (BIDS-Net), integrating CNN and Transformer models to enhance inpainting quality by leveraging their complementary strengths.

## Methodology Overview
1. Dual-Stream Structure:

- CNN Stream: Captures rich local patterns and refines details.
- Transformer Stream: Models long-range contextual correlations for global information.
- Both streams are based on a U-shaped encoder-decoder structure to facilitate efficient multi-scale context reasoning.

2. Bidirectional Feature Interaction (BFI):

Implements **bidirectional feature alignment and fusion** between the CNN and Transformer streams.
Employs **Selective Feature Fusion (SFF)** for adaptive feature integration by learning channel weights.
3. Fast Global Self-Attention:

Utilizes a **kernelizable fast-attention mechanism** for the Transformer, reducing computational complexity to linear.
4. Loss Functions:

Combines pixel-wise reconstruction, adversarial, perceptual, and style losses to ensure inpainting quality and perceptual consistency.

- Channel allocation: Optimal performance when CNN and Transformer streams have equal importance.
- Fusion methods: Bidirectional fusion outperforms unidirectional and unified-path approaches.
- Specific fusion techniques: SFF surpasses element-wise addition and concatenation.
- Number of random features: Optimal trade-off achieved with 72 orthogonal random features.